In [1]:
pwd

'/root/hald_autoupdate/2022_12_09'

In [2]:
import os
import pandas as pd

import csv
import pickle
import json


In [3]:
pd.set_option('display.max_columns', 50)

In [4]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [5]:
len(jcr)

21430

In [6]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
21420,12425,AGENDA,AGENDA,2022,NaN,0002-0796,159,0.0,0.0,0.0,0.0,0,17.8,0.0,0.00005,0.114,NaN,NaN,NaN,NaN,NaN,0.0,0.01206
21421,12425,Abakos,ABAKOS,2022,NaN,2316-9451,7,0.0,0.0,0.0,0.0,18,0.0,10.7,0.00003,0.068,NaN,NaN,NaN,NaN,NaN,0.0,0.00733
21422,12425,Adalya,ADALYA,2022,NaN,1301-2746,75,0.0,0.0,0.0,0.0,19,0.0,16.2,0.00027,0.306,NaN,NaN,NaN,NaN,NaN,0.0,0.05846
21423,12425,Anduli,ANDULI,2022,NaN,1696-0270,35,0.0,0.0,0.0,0.0,15,0.0,14.6,0.00007,0.150,NaN,NaN,NaN,NaN,NaN,0.0,0.01555
21424,12425,Anemia,ANEMIA,2022,NaN,2090-1267,502,0.0,0.0,0.0,0.0,14,7.3,7.7,0.00030,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.06589
21425,12425,Appeal,APPEAL,2022,NaN,1205-612X,56,0.0,0.0,0.0,0.0,6,0.0,4.8,0.00019,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.04134
21426,12425,Atenas,ATENAS,2022,NaN,1682-2749,60,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00007,0.034,NaN,NaN,NaN,NaN,NaN,0.0,0.01561
21427,12425,Atenea,ATENEA-CONCEPC,2022,NaN,0718-0462,105,0.0,0.0,0.0,0.0,35,8.8,15.8,0.00034,0.281,NaN,NaN,NaN,NaN,NaN,0.0,0.07463
21428,12425,AusArt,AUSART,2022,NaN,2340-9134,13,0.0,0.0,0.0,0.0,32,0.0,11.8,0.00002,0.017,NaN,NaN,NaN,NaN,NaN,0.0,0.00594


In [7]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [8]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [9]:
list(jt2if_brief.items())[:2]

[('ca-cancer j clin', 286.13), ('lancet', 202.731)]

In [10]:
len(jt2if_brief)

21429

In [11]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [12]:
jt2if5_brief={}
for i in range(len(jcr)):
    
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [13]:
list(jt2if5_brief.items())[:2]

[('ca-cancer j clin', 334.259), ('lancet', 130.838)]

In [14]:
import pymongo 

In [15]:
from pymongo import MongoClient

In [16]:
client = MongoClient(username="hald",password="ddd")

In [17]:
db = client.hald

In [18]:
collections_pubtator_info = db.pubtator_info

In [19]:
pubtator_info=list(collections_pubtator_info.find())

In [20]:
len(pubtator_info)

170387

In [21]:
pubtator_info[10]

{'_id': ObjectId('6393478a5303aaecac2745e7'),
 'PMID': '35794516',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20220708',
 'LR': '20220708',
 'IS': '1471-227X (Electronic) 1471-227X (Linking)',
 'VI': '22',
 'IP': '1',
 'DP': '2022 Jul 6',
 'TI': 'Long-term prognosis and clinical course of choking-induced cardiac arrest in patients without the return of spontaneous circulation at hospital arrival: a population-based community study from the Shizuoka Kokuho Database.',
 'PG': '120',
 'LID': '10.1186/s12873-022-00676-8 [doi]',
 'AB': 'BACKGROUND: The risk of choking increases with aging, and the number of cases of choking-induced cardiac arrest is increasing. However, few studies have examined the prognosis of choking-induced cardiac arrest. The aim of this study was to reveal the rates of survival and dependence on devices in the long term after choking-induced cardiac arrest. METHODS: We analyzed data from the Shizuoka Kokuho Database, which consists of claims data of approximately 2.

In [22]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [23]:
triplelist=[]

In [24]:
triplelist=json.load(open('triplelist.json'))

In [25]:
for i in triplelist:
    if i.get('target1_type')=='DNA':
        i['target1_type']='Gene'
    if i.get('target2_type')=='DNA':
        i['target2_type']='Gene'

In [26]:
triplelist[0]

{'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway r

In [27]:
# 122534
len(triplelist)

122534

In [28]:
triplelist[0]

{'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway r

In [29]:
be=['am','is','are','was','were','been','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'


In [30]:
triplelist[10].get('networkx')[1].split(' ')

['investigated']

In [31]:
for i in triplelist:
    if i.get('allennlp_final') and len(i.get('allennlp_final'))>1:
        print(i)
        break

{'PMID': '35955698', 'entity1': 'LRG1', 'entity2': 'LEP', 'sentence': 'In this study, we investigated the association of LRG1 with obesity markers, including leptin and other adipokines in adolescents (11-14 years; n = 425).', 'AB': 'Leucine-rich alpha-2 glycoprotein1 (LRG1) is a member of the leucine-rich repeat (LRR) family that is implicated in multiple diseases, including cancer, aging, and heart failure, as well as diabetes and obesity. LRG1 plays a key role in diet-induced hepatosteatosis and insulin resistance by mediating the crosstalk between adipocytes and hepatocytes. LRG1 also promotes hepatosteatosis by upregulating de novo lipogenesis in the liver and suppressing fatty acid beta-oxidation. In this study, we investigated the association of LRG1 with obesity markers, including leptin and other adipokines in adolescents (11-14 years; n = 425). BMI-for-age classification based on WHO growth charts was used to define obesity. Plasma LRG1 was measured by ELISA, while other mark

In [32]:
'''将entity_one_transformed和entity_two_transformed的列表外套脱掉，筛掉同篇文献同一个实体对，筛掉两个单词为同一实体'''
import copy
triplelist_takeoff=[]
transformed_triples=set()
box=[]
box2=[]
acount=0
for i in triplelist:
    maxlength=[]
    allennlp_list=[]
    if i.get('openie_final'):
        maxlength=i.get('openie_final')[0]
        for j in i.get('openie_final'):
            if len(j[1])>len(maxlength[1]):
                maxlength=j
        mid_list=maxlength[1].split(' ')
        for p,q in enumerate(mid_list):
            if bedict.get(q):
                mid_list[p]='be'
        maxlength[1]=' '.join(mid_list)
        for k in maxlength[1]:
            if k in maxlength[0].split(' ') or k in maxlength[2].split(' '):
                break
        else:
            mid=tuple([maxlength[0],maxlength[1],maxlength[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if maxlength[1]=='associated' or maxlength[1]=='associate' :
                mid=tuple([maxlength[2],maxlength[1],maxlength[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
        for j in i.get('openie_final'):
            if j==maxlength:
                continue
            judge=True
            for k in j[1].split(' '):
                if k not in maxlength[1].split(' '):
                    judge=False
                    break
            if not judge:
                mid_list=j[1].split(' ')
                for p,q in enumerate(mid_list):
                    if bedict.get(q):
                        mid_list[p]='be'
                j[1]=' '.join(mid_list)
                mid=tuple([j[0],j[1],j[2],i.get('PMID')])
                for k in j[1]:
                    if k in j[0].split(" ") or k in j[1].split(" "):
                        break
                else:
                    if mid not in transformed_triples:
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                                  'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                                  'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    if j[1]=='associated' or j[1]=='associate' :
                        mid=tuple([j[2],j[1],j[0],i.get('PMID')])
                        if mid not in transformed_triples:
                            copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                      'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                      'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                      'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                        'transformed_triple':mid}
                            triplelist_takeoff.append(copydict)
                            transformed_triples.add(mid)
    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            if len(maxlength)>0 and j[1] in maxlength[1].split(' '):
                pass
            else:
                mid=tuple([j[0],j[1],j[2],i.get('PMID')])
                if j[1] in j[0].split(' ') or j[1] in j[2].split(' '):
                    continue
                if mid not in transformed_triples:
                    allennlp_list.append((j[0],j[2]))
                    copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                              'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                              'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
                else:
                    box2.append(i)
                if j[1]=='associated' or j[1]=='associate' :
                    mid=tuple([j[2],j[1],j[0],i.get('PMID')])
                    if mid not in transformed_triples:
                        allennlp_list.append([j[2],j[0]])
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                  'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                  'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    else:
                        box2.append(i)
    if i.get('networkx'):
        if len(maxlength)>0 and i.get('networkx')[1] in maxlength[1].split(' '):
            box.append(i)
            pass
        else:
            # for j in allennlp_list:
            #     if i.get('networkx')[0]==j[1] and i.get('networkx')[1]==j[0]:
            #         print(i.get('networkx'))
            #         print(i)
            if i.get('networkx')[1] in i.get('networkx')[0].split(' ') or i.get('networkx')[1] in i.get('networkx')[2].split(' '):
                acount+=1
                continue
            mid=tuple([i.get('networkx')[0],i.get('networkx')[1],i.get('networkx')[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
                mid=tuple([i.get('networkx')[2],i.get('networkx')[1],i.get('networkx')[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)



In [33]:
len(box)

1363

In [34]:
len(box2)

1771

In [35]:
mid

('cardiovascular disease', 'associated', 'rs5882', '17531380')

In [36]:
print(len(triplelist_takeoff))
print(len(transformed_triples))

143005
143005


#114062
#114062
print(len(triplelist_takeoff))
print(len(transformed_triples))

'''将entity_one_transformed和entity_two_transformed的列表外套脱掉，筛掉同篇文献同一个实体对，筛掉两个单词为同一实体'''
import copy
triplelist_takeoff=[]
transformed_triples=set()
for i in triplelist:
    openie_final_list=[]
    if i.get('openie_final'):
        for j in i.get('openie_final'):
            
            mid=tuple([j[0],j[1],j[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if j[1]=='associated' or j[1]=='associate' :
                mid=tuple([j[2],j[1],j[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
    if i.get('networkx'):
        # mid_list=i.get('networkx')[1].split(' ')
        # for p,q in enumerate(mid_list):
        #     if bedict.get(q):
        #         mid_list[p]='be'
        # i['networkx'][1]=' '.join(mid_list)
        mid=tuple([i.get('networkx')[0],i.get('networkx')[1],i.get('networkx')[2],i.get('PMID')])
        if mid not in transformed_triples:
            copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                      'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                      'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                      'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                        'transformed_triple':mid}
            triplelist_takeoff.append(copydict)
            transformed_triples.add(mid)
        if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
            mid=tuple([i.get('networkx')[2],i.get('networkx')[1],i.get('networkx')[0],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                          'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                          'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)

    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            # mid_list=j[1].split(' ')
            # for p,q in enumerate(mid_list):
            #     if bedict.get(q):
            #         mid_list[p]='be'
            # j[1]=' '.join(mid_list)
            mid=tuple([j[0],j[1],j[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('entity1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('entity2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('target1'),'entity_two_transformed':i.get('target2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if j[1]=='associated' or j[1]=='associate' :
                mid=tuple([j[2],j[1],j[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)

for i in triplelist:
    if i.get('PMID')=='23396152' and i.get('entity1')=='ADIPOQ':
        print(i)

'''将entity_one_transformed和entity_two_transformed的列表外套脱掉，筛掉同篇文献同一个实体对，筛掉两个单词为同一实体'''
import copy
triplelist_takeoff=[]
transformed_triples=set()
for i in triplelist:
    for j in i.get('entity_one_transformed'):
        for k in i.get('entity_two_transformed'):
            if j==k:
                continue
            else:
                mid=(j,i.get('verb'),k,i.get('PMID'))
                if mid not in transformed_triples:
                    copydict=copy.deepcopy(i)
                    copydict.update({'entity_one_transformed':j,'entity_two_transformed':k,'transformed_triple':mid})
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)

In [37]:
# 114062
len(triplelist_takeoff)

143005

In [38]:
import re
triplelist_takesoff=[]
for i in triplelist_takeoff:
    if not re.search(r'\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\{|\[|\}|\]|\;|\:|\'|\"|\,|\<|\.|\>|\/|\?',i.get('triple')[1]):
        if not re.search(r'(\s\d+\s)|(\b\d+\b)',i.get('triple')[1]): 
            triplelist_takesoff.append(i)  
        

for i in triplelist_takeoff:
    if i.get('triple')=='4':
        print(i)

cxz=0
for i in triplelist:
    if i.get('networkx') and i.get('allennlp_final'):
        if i.get('networkx')[1]!=i.get('allennlp')[0][1]:
            cxz+=1
            if cxz<30:
                print(i)
            else:
                break

In [39]:
triplelistnewcopy=copy.deepcopy(triplelist_takesoff)

initial_count=0
for i in triplelistnewcopy:
    if re.search(r'\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\{|\[|\}|\]|\;|\:|\'|\"|\,|\<|\.|\>|\/|\?',i.get('triple')[1]):
        initial_count+=1
        # print(i.get('triple'))
        # if initial_count>20:
        #     break
    elif re.search(r'(\s\d+\s)|(\b\d+\b)',i.get('triple')[1]):
        print(i.get('triple')+[i.get('sentence')])

initial_count

deprecated_relations=pd.read_csv('classofrelations/deprecated_relations.csv',header=None,names=['word'])

In [40]:
deprecated_relations=pd.read_csv('classofrelations/deprecated.csv',header=None,names=['word'])

In [41]:
len(deprecated_relations)

208

In [42]:
deprecated_relations.head()

,word
0,% for
1,'s III be
2,)evaluate
3,#NAME?
4,#NAME?


In [43]:
deprecated_relations_dict={}
for i in deprecated_relations['word'].to_list():
    deprecated_relations_dict[i.lower()]=True

In [44]:
list(deprecated_relations_dict.items())[20]

('aforemention', True)

In [45]:
triplelistnewcopy[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [46]:
i

'vaccinated'

In [47]:
triplelistnew=[]
for i in triplelistnewcopy:
    if not deprecated_relations_dict.get(i.get('triple')[1].lower()):
        triplelistnew.append(i)

In [48]:
'''
含有重复的triplelistnew数量为56723
'''

'\n含有重复的triplelistnew数量为56723\n'

In [49]:
len(triplelistnew)

133446

In [50]:
triplelistnew[10]

{'triple': ['rs1001179', 'investigated', 'cataract'],
 'PMID': '35556207',
 'sentence': 'Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.',
 'AB': 'Hyperglycemia plays a pronounced role in accelerating the process of aging due to high oxidative stress which triggers dyslipidemia and subsequently led to the progression of cataract. The aim of this study was to investigate lipid profile and its relationship with genotypes of SOD1, GPX1, and CAT variants in cataract patients. Total n = 680 samples were screened in four groups: senile cataract (SC), diabetic cataract (DC), type 2 diabetes mellitus (DM), and controls (CL). Lipid profile was estimated and compared between groups, and its correlation was tested with glycemic markers. Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated i

In [51]:
f1=open(r'triple.txt','a')

In [52]:
for i in triplelistnew:
    for j,k in enumerate(i.get('transformed_triple')):
        if j==2:
            f1.write(k+'\n')
        elif j==0 or j==1:
            f1.write(k+'\t')


In [53]:
f1.close()

In [54]:

h_r_t_name = [":START_ID", "role", ":END_ID"]


In [55]:
h_r_t = pd.read_csv('triple.txt',sep='\t',names=h_r_t_name)

In [56]:
# 114062
len(h_r_t)

133446

In [57]:
# 去除重复实体 22849

entity = set()

entity_h = h_r_t[':START_ID'].tolist()

entity_t = h_r_t[':END_ID'].tolist()

for i in entity_h:

    entity.add(i)

for i in entity_t:

    entity.add(i)

print(len(entity))

23957


In [58]:
entity = list(entity)

In [59]:
'''
整合所有实体可能存在的类型 
entitydict就是实体转类型（分号相隔）
'''
import re
entitydict={}
for i,j in enumerate(triplelistnew):
    # if j.get('entity_one')==j.get('entity_two'):
    #     continue
    if not entitydict.get(j.get('entity_one_transformed')) and not entitydict.get(j.get('entity_two_transformed')):
        entitydict[j.get('entity_one_transformed')]=j.get('entity_one_type')
        entitydict[j.get('entity_two_transformed')]=j.get('entity_two_type')
    else:
        if entitydict.get(j.get('entity_one_transformed')):
            if re.search(j.get('entity_one_type'), entitydict.get(j.get('entity_one_transformed'))) :
                pass
            else:
                entitydict[j.get('entity_one_transformed')]+=";"+j.get('entity_one_type')
        else:
            entitydict[j.get('entity_one_transformed')]=j.get('entity_one_type')
        if entitydict.get(j.get('entity_two_transformed')):
            if re.search(j.get('entity_two_type'), entitydict.get(j.get('entity_two_transformed'))) :
                pass
            else:
                entitydict[j.get('entity_two_transformed')]+=";"+j.get('entity_two_type')
        else:
            entitydict[j.get('entity_two_transformed')]=j.get('entity_two_type')

In [60]:
# 5862
len(entitydict)

6181

In [61]:
list(entitydict.items())[-30:-1]

[('rs764254110', 'Mutation'),
 ('rs10137185', 'Mutation'),
 ('rs1256059', 'Mutation'),
 ('rs4988235', 'Mutation'),
 ('rs696217', 'Mutation'),
 ('rs3918242', 'Mutation'),
 ('rs401681', 'Mutation'),
 ('rs9272461', 'Mutation'),
 ('rs2004776', 'Mutation'),
 ('rs10824026', 'Mutation'),
 ('rs2971609', 'Mutation'),
 ('rs121918339', 'Mutation'),
 ('rs7503726', 'Mutation'),
 ('rs12252', 'Mutation'),
 ('rs2970848', 'Mutation'),
 ('rs6821591', 'Mutation'),
 ('rs2208454', 'Mutation'),
 ('rs79658334', 'Mutation'),
 ('rs28365927', 'Mutation'),
 ('Myopathy, Central Core', 'Disease'),
 ('rs12425791', 'Mutation'),
 ('rs188286943', 'Mutation'),
 ('rs3810950', 'Mutation'),
 ('rs7078012', 'Mutation'),
 ('rs17081935', 'Mutation'),
 ('rs2076260', 'Mutation'),
 ('rs3027178', 'Mutation'),
 ('rs1801020', 'Mutation'),
 ('rs1801155', 'Mutation')]

In [62]:
for i,j in entitydict.items():
    if re.search(';',j):
        print({i:j})


{'Ceramides': 'Lipid;Carbohydrate'}
{'Bleomycin': 'Carbohydrate;Peptide'}
{'Lipopolysaccharides': 'Carbohydrate;Toxin'}


for i in triplelist:
    if i.get('entity1')=='Ceramides' and i.get('target1_type')=='Carbohydrate':
        print(i)
        break
    elif i.get('entity2')=='Ceramides' and i.get('target2_type')=='Carbohydrate':
        print(i)
        break

In [63]:
list(entitydict.items())[:20]

[('SF3B1', 'Gene'),
 ('EIF2AK1', 'Gene'),
 ('Protoporphyria, Erythropoietic', 'Disease'),
 ('SOD1', 'Gene'),
 ('GPX1', 'Gene'),
 ('Cataract', 'Disease'),
 ('CAT', 'Gene'),
 ('rs1800668', 'Mutation'),
 ('Inflammation', 'Disease'),
 ('Seizures', 'Disease'),
 ('rs1001179', 'Mutation'),
 ('CDKN1A', 'Gene'),
 ('EGR1', 'Gene'),
 ('CPT1A', 'Gene'),
 ('TP53', 'Gene'),
 ('Fatty Acids', 'Lipid'),
 ('ACAN', 'Gene'),
 ('COL11A1', 'Gene'),
 ('WNT3', 'Gene'),
 ('Sclerosis', 'Disease')]

'''标注entity序号ID'''
entitynum={}
for i in entitydict.keys():
    entitynum[i]=i

In [64]:
'''标注entity序号ID'''
entitynum={}
num=1
for i in entitydict.keys():
    entitynum[i]=num
    num+=1

In [65]:
entitynum.get('APOE')

849

for i,j in entitynum.items():
    if j==768:
        print(i)

In [66]:
for i in triplelistnew:
    i['verb']=i.get('triple')[1]

# 9481
num

In [67]:
'''分别计算entity出现次数，relation出现次数和triple出现次数'''
entity_countnum={}
relation_countnum={}
tripleWeight={}
tripleWeightunique=set()
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if (i.get('entity_one_transformed'),i.get('triple')[1],i.get('entity_two_transformed'),i.get('PMID')) not in tripleWeightunique:
            tripleWeightunique.add((i.get('entity_one_transformed'),i.get('triple')[1],i.get('entity_two_transformed'),i.get('PMID')))
            entity_countnum[i.get('entity_one_transformed')]=entity_countnum.get(i.get('entity_one_transformed'),0)+1
            entity_countnum[i.get('entity_two_transformed')]=entity_countnum.get(i.get('entity_two_transformed'),0)+1
            relation_countnum[i.get('triple')[1]]=relation_countnum.get(i.get('triple')[1],0)+1
            tripleWeight[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]=tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')),0)+1

In [68]:
# 51551
len(tripleWeight)

99033

In [69]:
len(triplelistnew)

133446

In [70]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [71]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re
def cal_date(i):
    j=i
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    return date


his=[]
for i in triplelistnew:
    if not jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()) and jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())!=0.0:
        if not jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()) and jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())!=0.0:
            his.append(i)

len(triplelistnew)

# 42529
len(his)

hiss=[]
for i in triplelistnew:
    if not jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()) and jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())!=0.0:
        if not jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()) and jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())!=0.0:
            hiss.append(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())

len(list(set(hiss)))

list(set(hiss))[:30]

jt2if_brief.get(pubtator_info_dict.get(his[100].get('PMID')).get('TA').lower())

In [72]:
for i in triplelistnew:
    if jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF']=jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF']=jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF5']=jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF5']=jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF5']=0.0

In [167]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [168]:
'''计算同一组triple分别在哪些文章那些句子中出现'''
relation_interpolation_list=[]
relation_interpolation_dict={}
for i in triplelistnew:
    if (i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')) not in relation_interpolation_dict.keys():
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]={'from':entitynum.get(i.get('entity_one_transformed'))-1,
                                   'to':entitynum.get(i.get('entity_two_transformed'))-1,'relation':i.get('triple')[1],
                                   'PMID':[i.get('PMID')],'sentence':[i.get('sentence')],
                                   'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
                                   'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
                                    'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
                                    'IF':[i.get('IF')],   'IF5':[i.get('IF5')],   
                                    'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
                                    'first_entity_transformed':i.get('entity_one_transformed'),'second_entity_transformed':i.get('entity_two_transformed'),                                                     
                                    'first_entity':[i.get('entity_one')],'second_entity':[i.get('entity_two')],
                                    'first_type':i.get('entity_one_type'),'second_type':i.get('entity_two_type')}

    else:
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['PMID']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('PMID')+[i.get('PMID')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['sentence']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('sentence')+[i.get('sentence')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['first_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('first_entity')+[i.get('entity_one')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['second_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('second_entity')+[i.get('entity_two')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['DP']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TI']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TA']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['date']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF')+[i.get('IF')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF5']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF5')+[i.get('IF5')]

'''计算同一组triple分别在哪些文章那些句子中出现'''
relation_interpolation_list=[]
relation_interpolation_dict={}
for i in triplelistnew:
    if (i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed')) not in relation_interpolation_dict.keys():
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))]={'from':entitynum.get(i.get('entity_one_transformed'))-1,
                                   'to':entitynum.get(i.get('entity_two_transformed'))-1,'relation':i.get('verb'),
                                   'PMID':[i.get('PMID')],'sentence':[i.get('sentence')],
                                    'first_entity_transformed':i.get('entity_one_transformed'),'second_entity_transformed':i.get('entity_two_transformed'),                                                     
                                    'first_entity':[i.get('entity_one')],'second_entity':[i.get('entity_two')]}
    else:
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))]['PMID']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))).get('PMID')+[i.get('PMID')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))]['sentence']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))).get('sentence')+[i.get('sentence')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))]['first_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))).get('first_entity')+[i.get('entity_one')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))]['second_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('verb'), i.get('entity_two_transformed'))).get('second_entity')+[i.get('entity_two')]

In [169]:
list(relation_interpolation_dict.items())[10]

(('rs1001179', 'investigated', 'Cataract'),
 {'from': 10,
  'to': 5,
  'relation': 'investigated',
  'PMID': ['35556207'],
  'sentence': ['Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.'],
  'DP': ['2022 Sep'],
  'TI': ['Clinical Association of Biochemical Variations Among Multilocus Genotypes of Antioxidant Enzymes with Susceptibility of Cataract in Hyperglycemia.'],
  'TA': ['Appl Biochem Biotechnol'],
  'IF': [3.094],
  'IF5': [2.973],
  'date': [20220901],
  'first_entity_transformed': 'rs1001179',
  'second_entity_transformed': 'Cataract',
  'first_entity': ['rs1001179'],
  'second_entity': ['cataract'],
  'first_type': 'Mutation',
  'second_type': 'Disease'})

In [170]:
relation_interpolation_list=list(relation_interpolation_dict.values())

In [171]:
for i in relation_interpolation_list:
    if i.get('from')==0:
        print(i.get('from'))

0
0
0
0
0


In [172]:
# 51551
len(relation_interpolation_list)

99033

c=0
for i in relation_interpolation_list:
    if len(i.get('first_entity'))!=1:
        c+=1
        if c<=10:
            print(i)
        else:
            break

In [79]:
'''一个entity对应所有可能的句子和PMID'''
entity2content={}
for i,j in enumerate(triplelistnew):
    if j.get('entity_one_transformed') in entity2content.keys():
        entity2content[j.get('entity_one_transformed')]['sentence']=entity2content.get(j.get('entity_one_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_one_transformed')]['PMID']=entity2content.get(j.get('entity_one_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_one_transformed')]['entity']=entity2content.get(j.get('entity_one_transformed')).get('entity')+[j.get('entity_one')]
        entity2content[j.get('entity_one_transformed')]['DP']=entity2content.get(j.get('entity_one_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_one_transformed')]['TI']=entity2content.get(j.get('entity_one_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_one_transformed')]['TA']=entity2content.get(j.get('entity_one_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_one_transformed')]['date']=entity2content.get(j.get('entity_one_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_one_transformed')]['IF']=entity2content.get(j.get('entity_one_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_one_transformed')]['IF5']=entity2content.get(j.get('entity_one_transformed')).get('IF5')+[j.get('IF5')]
    else:
        entity2content[j.get('entity_one_transformed')]={'entity_transformed':j.get('entity_one_transformed'),'entity':[j.get('entity_one')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')]}
    if j.get('entity_two_transformed') in entity2content.keys():
        entity2content[j.get('entity_two_transformed')]['sentence']=entity2content.get(j.get('entity_two_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_two_transformed')]['PMID']=entity2content.get(j.get('entity_two_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_two_transformed')]['entity']=entity2content.get(j.get('entity_two_transformed')).get('entity')+[j.get('entity_two')]
        entity2content[j.get('entity_two_transformed')]['DP']=entity2content.get(j.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_two_transformed')]['TI']=entity2content.get(j.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_two_transformed')]['TA']=entity2content.get(j.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_two_transformed')]['date']=entity2content.get(j.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_two_transformed')]['IF']=entity2content.get(j.get('entity_two_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_two_transformed')]['IF5']=entity2content.get(j.get('entity_two_transformed')).get('IF5')+[j.get('IF5')]
    else:
        entity2content[j.get('entity_two_transformed')]={'entity_transformed':j.get('entity_two_transformed'),'entity':[j.get('entity_two')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')]}

In [80]:
# 9480
len(entity2content)

6181

list(entity2content.items())[1041]

In [81]:
entity2list=list(entity2content.values())

entity2list[1001]

In [82]:
len(entity2list)

6181

In [83]:
for i in entity2list:
    i['_id']=i.get('entity_transformed')

In [84]:
entity2list[1001]

{'entity_transformed': 'WNT1',
 'entity': ['Wnt1',
  'Wnt1',
  'WNT1',
  'Wnt1',
  'Wnt1',
  'WNT1',
  'WNT1',
  'Wnt1',
  'Wnt1',
  'Wnt1'],
 'sentence': ['In HKC-8 cells, Klotho significantly inhibited Wnt1- and Wnt9a-induced mitochondrial injury, cellular senescence, and fibrotic lesions.',
  'Quantitative real-time PCR shown that Wnt1, Wnt5a, Wnt7a, and Wnt9a mRNAs were lower expressed in osteoporosis derived EVs.',
  'Loss-of-function variants in LRP5 and WNT1 may lead to early-onset osteoporosis.',
  'Besides, Klotho caused downregulation of Wnt1 level, upregulation of phosphorylated cyclic AMP response element binding (pCREB), and mRNA levels of nuclear factor erythroid 2-related factor 2 (Nrf2) and heme oxygenase 1 (HO-1) with no significant alteration of epsilon isoform of protein kinase C (PKCepsilon) after Abeta toxicity.',
  'Besides, Klotho caused downregulation of Wnt1 level, upregulation of phosphorylated cyclic AMP response element binding (pCREB), and mRNA levels of nu

In [85]:
for i in entity2list:
    i['type']=entitydict.get(i.get('entity_transformed')).split(';')

In [86]:
entity2list[10]

{'entity_transformed': 'rs1001179',
 'entity': ['rs1001179'],
 'sentence': ['Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.'],
 'PMID': ['35556207'],
 'DP': ['2022 Sep'],
 'TI': ['Clinical Association of Biochemical Variations Among Multilocus Genotypes of Antioxidant Enzymes with Susceptibility of Cataract in Hyperglycemia.'],
 'TA': ['Appl Biochem Biotechnol'],
 'date': [20220901],
 'IF': [3.094],
 'IF5': [2.973],
 '_id': 'rs1001179',
 'type': ['Mutation']}

In [87]:
import json
json.dump(entity2list,open('entityinfo.json','w'))

In [88]:
list(relation_interpolation_dict.items())[:3]

[(('SF3B1', 'lead', 'EIF2AK1'),
  {'from': 0,
   'to': 1,
   'relation': 'lead',
   'PMID': ['35926182'],
   'sentence': ['Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.'],
   'DP': ['2022 Nov 2'],
   'TI': ['Targeting the EIF2AK1 Signaling Pathway Rescues Red Blood Cell Production in SF3B1-Mutant Myelodysplastic Syndromes With Ringed Sideroblasts.'],
   'TA': ['Blood Cancer Discov'],
   'IF': [0.0],
   'IF5': [0.0],
   'date': [20221102],
   'first_entity_transformed': 'SF3B1',
   'second_entity_transformed': 'EIF2AK1',
   'first_entity': ['SF3B1'],
   'second_entity': ['EIF2AK1']}),
 (('SF3B1', 'lead', 'Protoporphyria, Erythropoietic'),
  {'from': 0,
   'to': 2,
   'relation': 

for i,j in relation_interpolation_dict.items():
    if j.get('from')==768 and j.get('to')==2214:
        print({i:j})

for i,j in relation_interpolation_dict.items():
    if j.get('first_entity_transformed')=='APOE' and j.get('second_entity_transformed')=='APOB':
        print({i:j})

In [173]:
relation_interpolation_list[10]

{'from': 10,
 'to': 5,
 'relation': 'investigated',
 'PMID': ['35556207'],
 'sentence': ['Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.'],
 'DP': ['2022 Sep'],
 'TI': ['Clinical Association of Biochemical Variations Among Multilocus Genotypes of Antioxidant Enzymes with Susceptibility of Cataract in Hyperglycemia.'],
 'TA': ['Appl Biochem Biotechnol'],
 'IF': [3.094],
 'IF5': [2.973],
 'date': [20220901],
 'first_entity_transformed': 'rs1001179',
 'second_entity_transformed': 'Cataract',
 'first_entity': ['rs1001179'],
 'second_entity': ['cataract'],
 'first_type': 'Mutation',
 'second_type': 'Disease'}

In [90]:
for i in relation_interpolation_list:
    if i.get('first_entity_transformed')=='APOE' and i.get('second_entity_transformed')=='APOB':
        print(i)

{'from': 848, 'to': 2447, 'relation': 'confirm', 'PMID': ['33948810'], 'sentence': ['The analysis replicated a protein signature associated with APOE genotypes and confirmed strong overexpression of BIRC2 (p < 5E-16) and under-expression of APOB in carriers of the APOE2 allele (p < 0.05).'], 'DP': ['2021 Jun'], 'TI': ['Effect of longevity genetic variants on the molecular aging rate.'], 'TA': ['Geroscience'], 'IF': [7.581], 'IF5': [7.617], 'date': [20210601], 'first_entity_transformed': 'APOE', 'second_entity_transformed': 'APOB', 'first_entity': ['APOE'], 'second_entity': ['APOB']}
{'from': 848, 'to': 2447, 'relation': 'associated', 'PMID': ['27755538'], 'sentence': ['In NPHSII, ApoE concentration was additionally associated with apolipoprotein B (correlation coefficient 0.13, p = 0.001) and lipoprotein(a) (correlation coefficient -0.11, p < 0.001).'], 'DP': ['2016 Oct'], 'TI': ['Circulating Apolipoprotein E Concentration and Cardiovascular Disease Risk: Meta-analysis of Results from 

In [91]:
len(relation_interpolation_list)

99033

In [174]:
json.dump(relation_interpolation_list,open('relationinfo.json','w'))

In [175]:
len(relation_interpolation_list)

99033

for i in relation_interpolation_list:
    if i.get('from')==25 and i.get('to')==2070:
        print(i)

In [93]:
len(triplelistnew)

133446

In [94]:
csvf_entity = open("entity.csv", "w", newline='', encoding='utf-8')

w_entity = csv.writer(csvf_entity)

In [95]:
w_entity.writerow(("entity:ID", "name","type","pagerank",":LABEL"))

37

In [96]:
for i,j in entitydict.items():
    
    w_entity.writerow((entitynum.get(i), i,j,int(entity_countnum.get(i)),j))


for i,j in enumerate(triplelist):
    w_entity.writerow((entitynum.get(j.get('entity_one')), j.get('entity_one'), entitydict.get(j.get('entity_one'))))
    w_entity.writerow((entitynum.get(j.get('entity_two')), j.get('entity_two'), entitydict.get(j.get('entity_two'))))


In [97]:
csvf_entity.close()

In [98]:
h_r_t[:1]

,:START_ID,role,:END_ID
0,SF3B1,lead,EIF2AK1


In [99]:
entitycsv=pd.read_csv('entity.csv')

In [100]:
entitycsv[:3]

,entity:ID,name,type,pagerank,:LABEL
0,1,SF3B1,Gene,5,Gene
1,2,EIF2AK1,Gene,3,Gene
2,3,"Protoporphyria, Erythropoietic",Disease,6,Disease


In [101]:
len(entitycsv)

6181

entityone=[]
for z in range(0,21156,2):
    entityone.append(entitycsv['entity:ID'].tolist()[z])

entitytwo=[]
for z in range(1,21156,2):
    entitytwo.append(entitycsv['entity:ID'].tolist()[z])

In [102]:
csvf_entity = open("roles.csv", "w", newline='', encoding='utf-8')

w_entity = csv.writer(csvf_entity)

In [103]:
w_entity.writerow((':START_ID', ':END_ID', "relation", "weight",":TYPE" ))

41

In [104]:
entitynotsame=[]
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if (i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')) not in entitynotsame:
            w_entity.writerow((  entitynum.get(i.get('entity_one_transformed')),entitynum.get(i.get('entity_two_transformed')), i.get('triple')[1],int(tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')))), i.get('triple')[1]   ))
        else:
            continue
        entitynotsame.append((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')))

In [105]:
entitynum.get('APOE')

849

In [106]:
csvf_entity.close()

In [107]:
tuple((1,2,3))

(1, 2, 3)

In [108]:
pwd

'/root/hald_autoupdate/2022_12_09'

In [109]:
import pandas as pd

In [110]:
roles=pd.read_csv("roles.csv")

In [111]:
roles.head()

,:START_ID,:END_ID,relation,weight,:TYPE
0,1,2,lead,1,lead
1,1,3,lead,1,lead
2,4,5,indicate,1,indicate
3,5,6,investigated,1,investigated
4,7,5,indicate,1,indicate


In [112]:
relation_list=roles.relation.to_list()

In [113]:
type(relation_list)

list

In [114]:
relation_list[:3]

['lead', 'lead', 'indicate']

In [115]:
len(relation_list)

99033

In [116]:
len(set(relation_list))

3322

dropout

for i in dropout:
    print(i.get('triple'))

relation_list=[]
disease_list=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
            relation_list.append(i.get('triple')[1].lower())
            disease_list.append(i)

In [117]:
relation_list=[]
disease_list=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                relation_list.append(i.get('triple')[1].lower())
                disease_list.append(i)

In [118]:
len(relation_list)

18210

In [119]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in disease_list:
    j=pubtator_info_dict.get(i.get('PMID')).get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['TA']=pubtator_info_dict.get(i.get('PMID')).get('TA')
    i['JT']=pubtator_info_dict.get(i.get('PMID')).get('JT')
    i['year']=int(str(date)[:4])
    # del i['_id']
    # i['_id']=i.get('entity_one_type')+'_'+i.get('entity_one_transformed')

In [120]:
disease_list[20]

{'triple': ['coronary artery disease', 'associated', 'GDNF'],
 'PMID': '35957925',
 'sentence': 'The Wilcoxon-Gehan test indicates that a higher probability of three-year CE completion was associated with an age >65 years (P = 0.006), coronary artery disease (CAD) (P = 0.008), 6-Minute Walk Test <432 m (P = 0.034), serum glucose >120 mg/dL (P = 0.047), serum cortisol >10 mug/dL (P = 0.011), leptin >=15 ng/mL (P = 0.018), P-selectin >=23 ng/mL (P = 0.006) and GDNF >=20 pg/mL (P = 0.004).',
 'AB': 'Background: Societal aging - as a global demographic phenomenon - shows no indication of abating. As a result, the problem of age-associated disability and related long-term care is emerging as a major public health challenge. It is important that methods for identifying older adults at risk of adverse outcomes are implemented early. Methods: The study group consisted of 145 individuals, 44.1% women, who were randomized from community-dwelling 60-74-year-old adults. A comprehensive geriatric a

In [121]:
len(disease_list)

18210

In [122]:
relation_list=[]
for i in disease_list:
    relation_list.append(i.get('triple')[1])

In [123]:
len(set(relation_list))

1562

In [124]:
relation_list_set=list(set(relation_list))

In [125]:
relation_list_set.sort()

for i in triplelistnew:
    if i.get('triple')[1].lower()=='with cad be':
        print(i)

In [126]:
df_relation_list_set = pd.DataFrame(relation_list_set)

In [127]:
df_relation_list_set[:6]

,0
0,By may serve as
1,Determine
2,Established adverse effects of be
3,Expression of be
4,Loss of be
5,Overexpression of be


In [128]:
df_relation_list_set.to_csv('relation_list_set_02_28.csv')

positive_relations=pd.read_csv('classofrelations/positive_relations.csv',header=None,names=['word'])

In [129]:
positive_relations=pd.read_csv('classofrelations/positive-relations.csv',header=None,names=['word'])

In [130]:

positive_relations_dict={}
for i in positive_relations['word'].to_list():
    positive_relations_dict[i.lower()]=True

negative_relations=pd.read_csv('classofrelations/negative_relations.csv',header=None,names=['word'])

In [131]:
negative_relations=pd.read_csv('classofrelations/negative-relations.csv',header=None,names=['word'])

In [132]:

negative_relations_dict={}
for i in negative_relations['word'].to_list():
    negative_relations_dict[i.lower()]=True

In [133]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

dropout=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
            if positive_relations_dict.get(i.get('triple')[1].lower()):
                i['partofspeech']='positive'
            elif negative_relations_dict.get(i.get('triple')[1].lower()):
                i['partofspeech']='negative'
            else:
                i['partofspeech']='none'
                dropout.append(i)

In [134]:
dropout=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='positive'
                elif negative_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='negative'
                else:
                    i['partofspeech']='none'
                    dropout.append(i)

In [135]:
len(dropout)

8949

In [136]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [137]:
names=locals()
for i in entity_pairs:
    names[i+'_typecount']=set()
for i in entity_pairs:
    for j in disease_list:
        if j.get('entity_one_type')==i.replace('_',' '):
            names[i+'_typecount'].add(j.get('entity_one_transformed'))

In [138]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_typecount']))))

Gene 2310
RNA 62
Carbohydrate 206
Lipid 274
Peptide 55
Pharmaceutical_Preparations 10
Protein 35
Toxin 4
Mutation 394
Disease 0


In [139]:
json.dump(disease_list,open('disease_relation.json','w'))

In [423]:
# Table 3
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_aging']=set()
    for i in disease_list:
        if i.get('partofspeech')=='positive' and i.get('entity_one_type')==j:
            names[j+'_aging'].add(i.get('entity_one_transformed'))
    print(j+'\t'+str(len(names[j+'_aging'])))
    

Gene	1130
RNA	36
Carbohydrate	56
Lipid	69
Peptide	20
Pharmaceutical Preparations	2
Protein	12
Toxin	1
Mutation	268
Disease	0


In [424]:
# Table 3
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_longevity']=set()
    for i in disease_list:
        if i.get('partofspeech')=='negative' and i.get('entity_one_type')==j:
            names[j+'_longevity'].add(i.get('entity_one_transformed'))
    print(j+'\t'+str(len(names[j+'_longevity'])))
    

Gene	328
RNA	10
Carbohydrate	46
Lipid	37
Peptide	10
Pharmaceutical Preparations	2
Protein	7
Toxin	0
Mutation	7
Disease	0


In [441]:
Genedict=json.load(open('../dict/Genedict.json'))

In [443]:
list(Genedict.items())[:10]

[('A1BG', ['A1BG']),
 ('A1CF', ['A1CF']),
 ('A2M', ['A2M']),
 ('A2ML1', ['A2ML1']),
 ('A3GALT2', ['A3GALT2']),
 ('A4GALT', ['A4GALT']),
 ('A4GNT', ['A4GNT']),
 ('AAAS', ['AAAS']),
 ('AACS', ['AACS']),
 ('AADAC', ['AADAC'])]

In [444]:
Genedict_upper={}
for i,j in Genedict.items():
    Genedict_upper[i.upper()]=j

In [418]:
num_of_aging=set()
num_of_longevity=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        num_of_aging.add(i.get('entity_one_transformed'))
    elif i.get('partofspeech')=='negative':
        num_of_longevity.add(i.get('entity_one_transformed'))
print(len(num_of_aging))
print(len(num_of_longevity))

1594
447


In [414]:
disease_entity_list=[]
disease_entity_triple=set()
disease_entity_dict={}
for i in disease_list:
    if i.get('partofspeech')=='positive':
        if i.get('entity_one_type')=='Gene':
            # if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')) not in disease_entity_triple:

            disease_entity_triple.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list.append(i)
            disease_entity_dict[i.get('entity_one_transformed')]=disease_entity_dict.get(i.get('entity_one_transformed'),0)+1

In [417]:
len(disease_entity_dict)

1130

In [415]:
disease_entity_dict_sorted = sorted(disease_entity_dict.items(), key=lambda x: x[1], reverse=True)

In [416]:
len(disease_entity_dict_sorted)

1130

In [261]:
for i in disease_entity_dict_sorted[:20]:
    print(i)

('INS', 427)
('CRP', 297)
('APOE', 227)
('KL', 191)
('MAPT', 151)
('SIRT1', 114)
('IL6', 97)
('ADIPOQ', 91)
('LMNA', 72)
('LEP', 70)
('ALB', 66)
('WRN', 66)
('CD4', 64)
('TP53', 51)
('BDNF', 43)
('CST3', 43)
('SHBG', 43)
('FGF23', 41)
('ACE', 39)
('SIRT6', 37)


In [484]:
genage=pd.read_csv('genage_human.csv')

In [485]:
genage[:2]

,GenAge ID,symbol,name,entrez gene id,uniprot,why
0,1,GHR,growth hormone receptor,2690,GHR_HUMAN,mammal
1,2,GHRH,growth hormone releasing hormone,2691,SLIB_HUMAN,mammal


In [486]:
len(genage)

307

In [649]:
genage_symbol=set()
genage_symbol_transfer={}
for i in genage['symbol'].tolist():
    if Genedict_upper.get(i.upper()):
        genage_symbol.add(Genedict_upper.get(i.upper())[0])
        if genage_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in genage_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            genage_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(genage_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            genage_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]
    else:
        print(i)

TERC


In [650]:
len(genage_symbol)

306

In [651]:
vcx=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol:
        vcx+=1

In [652]:
vcx

156

In [653]:
vcx0=0
for i in disease_entity_dict_sorted[:10]:
    if i[0] in genage_symbol:
        vcx0+=1
print(vcx0)

8


In [336]:
disease_entity_list2=[]
disease_entity_triple2=set()
disease_entity_dict2={}
for i in disease_list:
    if i.get('partofspeech')=='negative':
        if i.get('entity_one_type')=='Gene':
            # if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')) not in disease_entity_triple:

            disease_entity_triple2.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list2.append(i)
            disease_entity_dict2[i.get('entity_one_transformed')]=disease_entity_dict2.get(i.get('entity_one_transformed'),0)+1

disease_entity_list3=[]
disease_entity_triple3=set()
disease_entity_dict3={}
for i in disease_list:
    if i.get('partofspeech')=='negative':
        if i.get('entity_one_type')=='Mutation':
            # if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')) not in disease_entity_triple:

            disease_entity_triple3.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list3.append(i)
            disease_entity_dict3[i.get('entity_one_transformed')]=disease_entity_dict3.get(i.get('entity_one_transformed'),0)+1

In [358]:
disease_entity_dict2_sorted = sorted(disease_entity_dict2.items(), key=lambda x: x[1], reverse=True)

disease_entity_dict3_sorted = sorted(disease_entity_dict3.items(), key=lambda x: x[1], reverse=True)

In [497]:
len(disease_entity_dict2_sorted)

328

len(disease_entity_dict3_sorted)

In [284]:
for i in disease_entity_dict2_sorted[:20]:
    print(i)

('SIRT1', 63)
('KL', 56)
('INS', 50)
('ACE', 24)
('SIRT6', 20)
('MAPT', 20)
('NFE2L2', 15)
('TP53', 15)
('APOE', 14)
('EPO', 14)
('PRKAA2', 11)
('MTOR', 11)
('SIRT3', 10)
('PTH', 10)
('CRP', 9)
('PRKAB1', 8)
('ADIPOQ', 8)
('NR3C2', 8)
('WRN', 7)
('AGER', 7)


In [498]:
longevity_human=pd.read_csv('longevity_human.csv')

In [499]:
longevity_human[:2]

,id,Association,Population,Variant(s),Gene(s),PubMed
0,1,non-significant,Dutch,HLA-B40,HLA-B,1859103
1,2,non-significant,Dutch,HLA-DRB5,HLA-DRB5,1859103


In [500]:
len(longevity_human)

550

In [656]:

longevity_human_symbol=set()
longevity_human_symbol_transfer={}
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            if Genedict_upper.get(i.upper()):
                longevity_human_symbol.add(Genedict_upper.get(i.upper())[0])
                if longevity_human_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in longevity_human_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
                    longevity_human_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
                    print(Genedict_upper.get(i.upper())[0])
                    print(longevity_human_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
                else:
                    longevity_human_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

longevity_human_mutation=set()
for j,i in enumerate(longevity_human['Variant(s)'].tolist()):
    if type(i)==str:
        for j in i.split(','):
            longevity_human_mutation.add(j)

In [657]:
#884
len(longevity_human_symbol)

766

len(longevity_human_mutation)

In [658]:
len(disease_entity_dict2_sorted)

328

In [659]:
vcx2=0
for i in disease_entity_dict2_sorted[:10]:
    if i[0] in longevity_human_symbol:
        vcx2+=1

In [660]:
vcx2

7

len(disease_entity_dict3_sorted)

In [661]:
vcx2=0
for i in disease_entity_dict2_sorted:
    if i[0] in longevity_human_symbol:
        vcx2+=1

In [662]:
#73
vcx2

75

aging_longevity_all=set()
for i in genage_symbol:
    aging_longevity_all.add(i)
for i in longevity_human_symbol:
    aging_longevity_all.add(i)

len(aging_longevity_all)

vcx4=0
for i in disease_entity_dict_sorted:
    if i[0] in aging_longevity_all:
        vcx4+=1
for i in disease_entity_dict2_sorted:
    if i[0] in aging_longevity_all:
        vcx4+=1

vcx4

In [605]:
gene_longevity_symbol_dict={}
# gene_longevity_symbol=[]
for i in disease_entity_dict_sorted:
    gene_longevity_symbol_dict[i[0]]=gene_longevity_symbol_dict.get(i[0],0)+i[1]
for i in disease_entity_dict2_sorted:
    gene_longevity_symbol_dict[i[0]]=gene_longevity_symbol_dict.get(i[0],0)+i[1]

In [606]:
len(gene_longevity_symbol_dict)

1232

In [607]:
gene_longevity_symbol = sorted(gene_longevity_symbol_dict.items(), key=lambda x: x[1], reverse=True)

In [609]:
gene_longevity_symbol[:2]

[('INS', 477), ('CRP', 306)]

In [663]:
vcx4=0
for i in gene_longevity_symbol:
    if i[0] in genage_symbol:
        vcx4+=1
print(vcx4)

164


In [664]:
vcx5=0
for i in gene_longevity_symbol:
    if i[0] in longevity_human_symbol:
        vcx5+=1
print(vcx5)

215


In [665]:
vcx44=0
for i in gene_longevity_symbol[:10]:
    if i[0] in genage_symbol:
        vcx44+=1
print(vcx44)

8


In [667]:
vcx55=0
for i in gene_longevity_symbol[:10]:
    if i[0] in longevity_human_symbol:
        vcx55+=1
print(vcx55)

8


276/884

In [521]:
156/306

0.5098039215686274

In [522]:
75/766

0.097911227154047

In [617]:
164/306

0.5359477124183006

In [618]:
215/766

0.28067885117493474

vcx3=0
for i in disease_entity_dict3_sorted:
    if i[0] in longevity_human_mutation:
        vcx3+=1

disease_entity_dict3_sorted

vcx3

for i in disease_entity_dict_sorted[:10]:
    if i[0] not in genage_symbol:
        print(i)

#Supplementary Table 1
data1=[]
data2=[]
data3=[]
for j,i in enumerate(gene_longevity_symbol):
    if i[0] in genage_symbol:
        data1.append(i[0])
        data2.append(i[1])
        data3.append(genage_symbol_transfer.get(i[0]))
dataframe = pd.DataFrame({'Genes':data1,'Frequency of occurrence':data2,'Corresponding to GenAge':data3})
dataframe.to_csv("./table4.csv", index=False)

#Supplementary Table 1
data1 = []
data2=[]
data3=[]
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol:
        data1.append(i[0])
        data2.append(i[1])
        data3.append(genage_symbol_transfer.get(i[0]))
dataframe = pd.DataFrame({'Genes':data1,'Frequency of occurrence':data2,'Corresponding to GenAge':data3})
dataframe.to_csv("./table4.csv", index=False)


#Supplementary Table 2
data4 = []
data5=[]
data6=[]
for i in disease_entity_dict2_sorted:
    if i[0] in longevity_human_symbol:
        data4.append(i[0])
        data5.append(i[1])
        data6.append(longevity_human_symbol_transfer.get(i[0]))
dataframe2 = pd.DataFrame({'Genes':data4,'Frequency of occurrence':data5,'Corresponding to LongevityMap':data6})
dataframe2.to_csv("./table5.csv", index=False)


In [619]:
print(len(longevity_human_symbol))

766


In [620]:
print(len(disease_entity_dict2_sorted))

328


In [432]:
#Figure 3
for i in disease_entity_dict_sorted[:20]:
    print(i[0])


INS
CRP
APOE
KL
MAPT
SIRT1
IL6
ADIPOQ
LMNA
LEP
ALB
WRN
CD4
TP53
BDNF
CST3
SHBG
FGF23
ACE
SIRT6


In [435]:
#Figure 3
for i in disease_entity_dict2_sorted[:20]:
    print(i[0])


SIRT1
KL
INS
ACE
SIRT6
MAPT
NFE2L2
TP53
APOE
EPO
PRKAA2
MTOR
SIRT3
PTH
CRP
PRKAB1
ADIPOQ
NR3C2
WRN
AGER


In [621]:
agingbank_genes_pro=pd.read_csv('agingbank_genes_pro.csv')

In [622]:
agingbank_genes_pro[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,ABI3,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
1,Homo sapiens,gene,ABI3BP,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
2,Homo sapiens,gene,ACE,human,NaN,NaN,Pro,16971231,details


In [623]:
agingbank_genes_pro_list=agingbank_genes_pro['name'].tolist()

In [624]:
len(agingbank_genes_pro_list)

97

In [625]:
# 发现有同一个实体对应不同的名字
agingbank_genes_pro_symbol=set()
agingbank_genes_pro_symbol_transfer={}
for i in agingbank_genes_pro_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        agingbank_genes_pro_symbol.add(Genedict_upper.get(i.upper())[0])
        if agingbank_genes_pro_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in agingbank_genes_pro_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(agingbank_genes_pro_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

IGF1
['IGF1', 'IGF-I']
CDKN2A
['CDKN2A', 'p16']
CDKN2A
['CDKN2A', 'p16', 'p16INK4a']
TP53
['p53', 'TP53']
CDKN2A
['p16INK4a', 'CDKN2A']
TP53
['TP53', 'p53']
CDKN2A
['CDKN2A', 'p16']


In [626]:
len(agingbank_genes_pro_symbol)

54

In [599]:
counts=[]
for i in agingbank_genes_pro_symbol:
    for j in disease_entity_dict_sorted[:10]:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

3


In [627]:
counts=[]
for i in agingbank_genes_pro_symbol:
    for j in disease_entity_dict_sorted:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

32


In [561]:
32/54

0.5925925925925926

In [628]:
vcx11=0
for i in gene_longevity_symbol:
    if i[0] in agingbank_genes_pro_symbol:
        vcx11+=1
print(vcx11)

33


In [630]:
33/54

0.6111111111111112

In [668]:
vcx66=0
for i in gene_longevity_symbol[:10]:
    if i[0] in agingbank_genes_pro_symbol:
        vcx66+=1
print(vcx66)

3


#Supplementary Table 3
data7 = []
data8=[]
data9=[]
for i in disease_entity_dict_sorted:
    if i[0] in agingbank_genes_pro_symbol:
        data7.append(i[0])
        data8.append(i[1])
        data9.append('; '.join(agingbank_genes_pro_symbol_transfer.get(i[0])))
dataframe2 = pd.DataFrame({'Genes':data7,'Frequency of occurrence':data8,'Corresponding to AgingBank':data9})
dataframe2.to_csv("./table6.csv", index=False)


In [631]:
agingbank_genes_anti=pd.read_csv('agingbank_genes_anti.csv')

In [632]:
agingbank_genes_anti[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,AEG-1,cell line,LNCaP;C42B,Down,Anti,28923415,details
1,Homo sapiens,gene,AKT1,human,Brain,NaN,Anti,16914869,details
2,Homo sapiens,gene,APOE,human,NaN,NaN,Anti,17234815,details


In [633]:
agingbank_genes_anti_list=agingbank_genes_anti['name'].tolist()

In [634]:
len(agingbank_genes_anti_list)

154

In [635]:
# 发现有同一个实体对应不同的名字
agingbank_genes_anti_symbol=set()
agingbank_genes_anti_symbol_transfer={}
for i in agingbank_genes_anti_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        agingbank_genes_anti_symbol.add(Genedict_upper.get(i.upper())[0])
        if agingbank_genes_anti_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in agingbank_genes_anti_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(agingbank_genes_anti_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

FOXO3
['FOXO3', 'FOXO3a']
PARP1
['PARP', 'PARP1']
STAG2
['SA2', 'STAG2']
TERT
['hEST2', 'TERT']
KL
['klotho', 'KL']


In [636]:
len(agingbank_genes_anti_symbol)

96

In [600]:
counts2=[]
for i in agingbank_genes_anti_symbol:
    for j in disease_entity_dict2_sorted[:10]:
        if i==j[0]:
            counts2.append(i)
print(len(counts2))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

6


In [571]:
counts2=[]
for i in agingbank_genes_anti_symbol:
    for j in disease_entity_dict2_sorted:
        if i==j[0]:
            counts2.append(i)
print(len(counts2))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

28


In [572]:
28/96

0.2916666666666667

In [637]:
vcx12=0
for i in gene_longevity_symbol:
    if i[0] in agingbank_genes_anti_symbol:
        vcx12+=1
print(vcx12)

45


In [638]:
45/96

0.46875

In [670]:
vcx77=0
for i in gene_longevity_symbol[:10]:
    if i[0] in agingbank_genes_anti_symbol:
        vcx77+=1
print(vcx77)

6


#Supplementary Table 4
data10 = []
data11=[]
data12=[]
for i in disease_entity_dict2_sorted:
    if i[0] in agingbank_genes_anti_symbol:
        data10.append(i[0])
        data11.append(i[1])
        data12.append('; '.join(agingbank_genes_anti_symbol_transfer.get(i[0])))
dataframe2 = pd.DataFrame({'Genes':data10,'Frequency of occurrence':data11,'Corresponding to AgingBank':data12})
dataframe2.to_csv("./table7.csv", index=False)


vcx6=0
for i in disease_entity_dict_sorted:
    if i[0] in agingbank_genes_pro_symbol:
        vcx6+=1
for i in disease_entity_dict2_sorted:
    if i[0] in agingbank_genes_pro_symbol:
        vcx6+=1

vcx6

49/54

vcx7=0
for i in disease_entity_dict_sorted:
    if i[0] in agingbank_genes_anti_symbol:
        vcx7+=1
for i in disease_entity_dict2_sorted:
    if i[0] in agingbank_genes_anti_symbol:
        vcx7+=1

vcx7

70/96

In [639]:
agingatlas_genes=pd.read_csv('agingatlas_genes.csv')

In [640]:
agingatlas_genes[:3]

,Symbol
0,IL2RB
1,PSAT1
2,A2M


In [641]:
agingatlas_genes_list=agingatlas_genes['Symbol'].tolist()

In [642]:
len(agingatlas_genes_list)

502

In [643]:
# 发现有同一个实体对应不同的名字
agingatlas_genes_symbol=set()
agingatlas_genes_symbol_transfer={}
for i in agingatlas_genes_list:
    if Genedict_upper.get(i.upper()):
        if Genedict_upper.get(i.upper())[0] in agingatlas_genes_symbol:
            print(Genedict_upper.get(i.upper())[0])
            print(i)
        agingatlas_genes_symbol.add(Genedict_upper.get(i.upper())[0])

        if agingatlas_genes_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in agingatlas_genes_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            agingatlas_genes_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(agingatlas_genes_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            agingatlas_genes_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]
    else:
        print(i)

RAD27
TERC


In [644]:
len(agingatlas_genes_symbol)

500

In [645]:
counts=[]
for i in agingatlas_genes_symbol:
    for j in disease_entity_dict_sorted[:10]:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)

9


In [646]:
counts=[]
for i in agingatlas_genes_symbol:
    for j in disease_entity_dict_sorted:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

222


In [597]:
222/500

0.444

#Supplementary Table 5
data13 = []
data14=[]
data15=[]
for i in disease_entity_dict_sorted:
    if i[0] in agingatlas_genes_symbol:
        data13.append(i[0])
        data14.append(i[1])
        data15.append('; '.join(agingatlas_genes_symbol_transfer.get(i[0])))
dataframe2 = pd.DataFrame({'Genes':data13,'Frequency of occurrence':data14,'Corresponding to AgingAtlas':data15})
dataframe2.to_csv("./table8.csv", index=False)


In [671]:
vcx88=0
for i in gene_longevity_symbol[:10]:
    if i[0] in agingatlas_genes_symbol:
        vcx88+=1
print(vcx88)

9


In [679]:
disease_entity_dict_sorted[:2]

[('INS', 427), ('CRP', 297)]

In [678]:
gene_longevity_symbol[:2]

[('INS', 477), ('CRP', 306)]

In [647]:
vcx13=0
for i in gene_longevity_symbol:
    if i[0] in agingatlas_genes_symbol:
        vcx13+=1
print(vcx13)

233


In [648]:
233/500

0.466

vcx8=0
for i in disease_entity_dict_sorted:
    if i[0] in agingatlas_genes_symbol:
        vcx8+=1
for i in disease_entity_dict2_sorted:
    if i[0] in agingatlas_genes_symbol:
        vcx8+=1

vcx8

319/500

In [689]:
#Supplementary Table 1

datas1= []
datas2=[]
datas3=[]
datas4=[]
datas5=[]
for i in disease_entity_dict_sorted:
    datas1.append(i[0])
    datas2.append(i[1])
    if genage_symbol_transfer.get(i[0]):
        datas3.append('; '.join(genage_symbol_transfer.get(i[0])))
    else:
        datas3.append('')
    if agingbank_genes_pro_symbol_transfer.get(i[0]):
        datas4.append('; '.join(agingbank_genes_pro_symbol_transfer.get(i[0])))
    else:
        datas4.append('') 
    if agingatlas_genes_symbol_transfer.get(i[0]):
        datas5.append('; '.join(agingatlas_genes_symbol_transfer.get(i[0])))
    else:
        datas5.append('') 
dataframe1 = pd.DataFrame({'HALD':datas1,'Frequency of occurrence':datas2,'GenAge':datas3,'AgingBank (Pro)':datas4,'Aging Atlas':datas5})
dataframe1.to_csv("./Supplementary1.csv", index=False)

In [690]:
#Supplementary Table 2

datas6= []
datas7=[]
datas8=[]
datas9=[]
datas10=[]
for i in disease_entity_dict2_sorted:
    datas6.append(i[0])
    datas7.append(i[1])
    if longevity_human_symbol_transfer.get(i[0]):
        datas8.append('; '.join(longevity_human_symbol_transfer.get(i[0])))
    else:
        datas8.append('')
    if agingbank_genes_anti_symbol_transfer.get(i[0]):
        datas9.append('; '.join(agingbank_genes_anti_symbol_transfer.get(i[0])))
    else:
        datas9.append('') 
dataframe2 = pd.DataFrame({'HALD':datas6,'Frequency of occurrence':datas7,'LongevityMap':datas8,'AgingBank (Anti)':datas9})
dataframe2.to_csv("./Supplementary2.csv", index=False)

In [691]:
#Supplementary Table 3

data1= []
data2=[]
data3=[]
data4=[]
data5=[]
data6=[]
data7=[]
data8=[]
for i in gene_longevity_symbol:
    label1=[]
    label2=[]
    data1.append(i[0])
    for j in disease_entity_dict_sorted:
        if i[0]==j[0]:
            label1.append('Aging')
            label2.append(str(j[1]))
            break
    for j in disease_entity_dict2_sorted:
        if i[0]==j[0]:
            label1.append('Longevity')
            label2.append(str(j[1]))
            break
    data2.append(' | '.join(label1))
    data3.append(' | '.join(label2))
    if genage_symbol_transfer.get(i[0]):
        data4.append('; '.join(genage_symbol_transfer.get(i[0])))
    else:
        data4.append('')
    if longevity_human_symbol_transfer.get(i[0]):
        data5.append('; '.join(longevity_human_symbol_transfer.get(i[0])))
    else:
        data5.append('')
    if agingbank_genes_pro_symbol_transfer.get(i[0]):
        data6.append('; '.join(agingbank_genes_pro_symbol_transfer.get(i[0])))
    else:
        data6.append('') 
    if agingbank_genes_anti_symbol_transfer.get(i[0]):
        data7.append('; '.join(agingbank_genes_anti_symbol_transfer.get(i[0])))
    else:
        data7.append('') 
    if agingatlas_genes_symbol_transfer.get(i[0]):
        data8.append('; '.join(agingatlas_genes_symbol_transfer.get(i[0])))
    else:
        data8.append('') 
dataframe3 = pd.DataFrame({'HALD':data1,'Aging or Longevity':data2,'Frequency of occurrence':data3,'GenAge':data4,'LongevityMap':data5,'AgingBank (Pro)':data6,'AgingBank (Anti)':data7,'Aging Atlas':data8})
dataframe3.to_csv("./Supplementary3.csv", index=False)

In [142]:
disease_entity_list=[]
disease_entity_triple=set()
for i in disease_list:
    if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('triple')[1]) not in disease_entity_triple:
        disease_entity_triple.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('triple')[1]))
        disease_entity_list.append(i)

In [143]:
len(disease_entity_list)

15430

ssss=[]
for i in range(2023,1958,-1):
    ssss.append(i)
print(ssss)

In [144]:
relation_interpolation_list[0]

{'from': 0,
 'to': 1,
 'relation': 'lead',
 'PMID': ['35926182'],
 'sentence': ['Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.'],
 'DP': ['2022 Nov 2'],
 'TI': ['Targeting the EIF2AK1 Signaling Pathway Rescues Red Blood Cell Production in SF3B1-Mutant Myelodysplastic Syndromes With Ringed Sideroblasts.'],
 'TA': ['Blood Cancer Discov'],
 'IF': [0.0],
 'IF5': [0.0],
 'date': [20221102],
 'first_entity_transformed': 'SF3B1',
 'second_entity_transformed': 'EIF2AK1',
 'first_entity': ['SF3B1'],
 'second_entity': ['EIF2AK1']}

In [145]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [394]:
direction=set()
direction_list=[]
direction_type=[]
for i in triplelistnew:
    if (i.get('entity_one_transformed'),(i.get('entity_two_transformed'))) not in direction:
        direction.add((i.get('entity_one_transformed'),(i.get('entity_two_transformed'))))
        direction_list.append((i.get('entity_one_transformed'),(i.get('entity_two_transformed'))))
        direction_type.append((i.get('entity_one_type').replace(' ','_'),(i.get('entity_two_type').replace(' ','_'))))

In [395]:
len(direction_list)

51203

In [396]:
type_dict={}
type_dict_ensure={}
for i in range(len(direction_list)):
    if not type_dict_ensure.get((direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])):
        type_dict[(direction_list[i][0],direction_list[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))
        type_dict_ensure[(direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))

In [397]:
len(type_dict)

51203

In [398]:
entity_pairs

['Gene',
 'RNA',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Protein',
 'Toxin',
 'Mutation',
 'Disease']

In [399]:
entity_pairs_dict={}
entity_pairs_dict_num={}
for i in entity_pairs:
    temp={}
    temp2={}
    for j in entity_pairs:
        temp.update({j:0})
        temp2.update({j:set()})
    entity_pairs_dict_num[i]=temp
    entity_pairs_dict[i]=temp2

In [400]:
entity_pairs_dict

{'Gene': {'Gene': set(),
  'RNA': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Protein': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'RNA': {'Gene': set(),
  'RNA': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Protein': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Carbohydrate': {'Gene': set(),
  'RNA': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Protein': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Lipid': {'Gene': set(),
  'RNA': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Protein': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Peptide': {'Gene': set(),
  'RNA': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptid

In [401]:
for i in range(len(direction_list)):
    if direction_list[i][0] not in entity_pairs_dict.get(direction_type[i][1]).get(direction_type[i][0]):
        entity_pairs_dict[direction_type[i][1]][direction_type[i][0]].add(direction_list[i][0])
        entity_pairs_dict_num[direction_type[i][1]][direction_type[i][0]]+=1
            

In [402]:
entity_pairs_dict_num

{'Gene': {'Gene': 1039,
  'RNA': 34,
  'Carbohydrate': 43,
  'Lipid': 43,
  'Peptide': 19,
  'Pharmaceutical_Preparations': 0,
  'Protein': 9,
  'Toxin': 1,
  'Mutation': 236,
  'Disease': 681},
 'RNA': {'Gene': 38,
  'RNA': 6,
  'Carbohydrate': 1,
  'Lipid': 0,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Protein': 0,
  'Toxin': 0,
  'Mutation': 1,
  'Disease': 35},
 'Carbohydrate': {'Gene': 125,
  'RNA': 1,
  'Carbohydrate': 58,
  'Lipid': 23,
  'Peptide': 7,
  'Pharmaceutical_Preparations': 1,
  'Protein': 3,
  'Toxin': 0,
  'Mutation': 6,
  'Disease': 264},
 'Lipid': {'Gene': 150,
  'RNA': 0,
  'Carbohydrate': 20,
  'Lipid': 67,
  'Peptide': 4,
  'Pharmaceutical_Preparations': 0,
  'Protein': 2,
  'Toxin': 1,
  'Mutation': 13,
  'Disease': 304},
 'Peptide': {'Gene': 49,
  'RNA': 0,
  'Carbohydrate': 7,
  'Lipid': 8,
  'Peptide': 7,
  'Pharmaceutical_Preparations': 1,
  'Protein': 1,
  'Toxin': 0,
  'Mutation': 1,
  'Disease': 88},
 'Pharmaceutical_Preparations': {'Gene': 

In [403]:
#8462
countnum=0
for i in entity_pairs_dict_num.values():
    for j in i.values():
        countnum+=j
print(countnum)

8453


In [404]:
relation_display=[]
for i,j in entity_pairs_dict_num.items():
    for k,l in j.items():
        relation_display.append([k,i+' ',l])

In [ ]:
relation_display

[['Gene', 'Gene ', 1039],
 ['RNA', 'Gene ', 34],
 ['Carbohydrate', 'Gene ', 43],
 ['Lipid', 'Gene ', 43],
 ['Peptide', 'Gene ', 19],
 ['Pharmaceutical_Preparations', 'Gene ', 0],
 ['Protein', 'Gene ', 9],
 ['Toxin', 'Gene ', 1],
 ['Mutation', 'Gene ', 236],
 ['Disease', 'Gene ', 681],
 ['Gene', 'RNA ', 38],
 ['RNA', 'RNA ', 6],
 ['Carbohydrate', 'RNA ', 1],
 ['Lipid', 'RNA ', 0],
 ['Peptide', 'RNA ', 0],
 ['Pharmaceutical_Preparations', 'RNA ', 0],
 ['Protein', 'RNA ', 0],
 ['Toxin', 'RNA ', 0],
 ['Mutation', 'RNA ', 1],
 ['Disease', 'RNA ', 35],
 ['Gene', 'Carbohydrate ', 125],
 ['RNA', 'Carbohydrate ', 1],
 ['Carbohydrate', 'Carbohydrate ', 58],
 ['Lipid', 'Carbohydrate ', 23],
 ['Peptide', 'Carbohydrate ', 7],
 ['Pharmaceutical_Preparations', 'Carbohydrate ', 1],
 ['Protein', 'Carbohydrate ', 3],
 ['Toxin', 'Carbohydrate ', 0],
 ['Mutation', 'Carbohydrate ', 6],
 ['Disease', 'Carbohydrate ', 264],
 ['Gene', 'Lipid ', 150],
 ['RNA', 'Lipid ', 0],
 ['Carbohydrate', 'Lipid ', 20],
 ['L

In [381]:
all_es=json.load(open('results/all_es_lack.json'))

In [382]:
entity2aging={}
entity2longevity={}
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_dict.get(i.get('triple')[1].lower()):
                    entity2aging[i.get('entity_one_transformed')]=True
                elif negative_relations_dict.get(i.get('triple')[1].lower()):
                    entity2longevity[i.get('entity_one_transformed')]=True

In [383]:
len(entity2aging)

1594

In [384]:
len(entity2longevity)

447

In [385]:
aaaa=0
for i in entity2aging.keys():
    if entity2longevity.get(i):
        aaaa+=1
print(aaaa)

316


In [386]:
for i in all_es:
    if entitydict.get(i.get('entity')):
        i['network']=True
    else:
        i['network']=False
    if entity2aging.get(i.get('entity')):
        i['aging']=True
    else:
        i['aging']=False
    if entity2longevity.get(i.get('entity')):
        i['longevity']=True
    else:
        i['longevity']=False

In [387]:
json.dump(all_es,open('results/all_es.json','w'))

In [388]:
len(all_es)

10907

In [140]:
type_dict2num={}
for i,j in type_dict.items():
    type_dict2num[j]=type_dict2num.get(j,0)+1

In [141]:
len(type_dict2num)

67

In [142]:
type_dict2num

{('Gene', 'Gene'): 2739,
 ('Gene', 'Disease'): 6290,
 ('Mutation', 'Disease'): 518,
 ('Disease', 'Disease'): 31816,
 ('Lipid', 'Gene'): 177,
 ('Disease', 'Gene'): 3816,
 ('Gene', 'Lipid'): 209,
 ('Lipid', 'Disease'): 934,
 ('Gene', 'Carbohydrate'): 155,
 ('Lipid', 'Lipid'): 249,
 ('Protein', 'Gene'): 30,
 ('Carbohydrate', 'Disease'): 677,
 ('Mutation', 'Carbohydrate'): 6,
 ('Disease', 'Lipid'): 672,
 ('Disease', 'Protein'): 91,
 ('Protein', 'Disease'): 127,
 ('Mutation', 'Mutation'): 167,
 ('Mutation', 'Gene'): 286,
 ('Disease', 'Mutation'): 375,
 ('Disease', 'Carbohydrate'): 498,
 ('Carbohydrate', 'Carbohydrate'): 140,
 ('Gene', 'RNA'): 43,
 ('RNA', 'Gene'): 56,
 ('Carbohydrate', 'Gene'): 139,
 ('Lipid', 'Carbohydrate'): 32,
 ('Gene', 'Protein'): 33,
 ('Gene', 'Peptide'): 56,
 ('Gene', 'Mutation'): 226,
 ('Disease', 'RNA'): 52,
 ('RNA', 'Disease'): 143,
 ('Protein', 'Carbohydrate'): 5,
 ('Mutation', 'Lipid'): 14,
 ('Lipid', 'Mutation'): 17,
 ('Peptide', 'Gene'): 59,
 ('Peptide', 'Carb

In [169]:
type_list=[]
for i,j in type_dict2num.items():
    type_list.append(list(i)+[j])

In [170]:
for i in range(len(type_list)):
    if type_list[i][0]==type_list[i][1]:
        type_list[i][1]='per se'

In [171]:
pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [172]:
pairs_sorted=sorted(pairs)+['per se']

In [173]:
type_list_two=[]
for i in type_list:
    type_list_two.append(i[:2])

In [174]:
type_list_mid=copy.deepcopy(type_list)
for i in range(len(pairs_sorted)-1):
    for j in list(range(0,i))+list(range(i+1,len(pairs_sorted))):
        if [pairs_sorted[i],pairs_sorted[j]] not in type_list_two:
            type_list_mid.append([pairs_sorted[i],pairs_sorted[j],0])

In [175]:
type_list_sorted=sorted(type_list_mid)

In [176]:
type_list_sorted

[['Carbohydrate', 'DNA', 111],
 ['Carbohydrate', 'Disease', 610],
 ['Carbohydrate', 'Lipid', 31],
 ['Carbohydrate', 'Mutation', 2],
 ['Carbohydrate', 'Peptide', 9],
 ['Carbohydrate', 'Pharmaceutical_Preparations', 0],
 ['Carbohydrate', 'Protein', 4],
 ['Carbohydrate', 'RNA', 0],
 ['Carbohydrate', 'Toxin', 0],
 ['Carbohydrate', 'per se', 127],
 ['DNA', 'Carbohydrate', 137],
 ['DNA', 'Disease', 5332],
 ['DNA', 'Lipid', 174],
 ['DNA', 'Mutation', 178],
 ['DNA', 'Peptide', 46],
 ['DNA', 'Pharmaceutical_Preparations', 0],
 ['DNA', 'Protein', 32],
 ['DNA', 'RNA', 11],
 ['DNA', 'Toxin', 17],
 ['DNA', 'per se', 2257],
 ['Disease', 'Carbohydrate', 432],
 ['Disease', 'DNA', 3079],
 ['Disease', 'Lipid', 532],
 ['Disease', 'Mutation', 272],
 ['Disease', 'Peptide', 104],
 ['Disease', 'Pharmaceutical_Preparations', 10],
 ['Disease', 'Protein', 78],
 ['Disease', 'RNA', 31],
 ['Disease', 'Toxin', 15],
 ['Disease', 'per se', 27994],
 ['Lipid', 'Carbohydrate', 31],
 ['Lipid', 'DNA', 153],
 ['Lipid', 'Di

In [ ]:
len(set(relation_list))

In [ ]:
len(entitynotsame)

In [ ]:
relation_list_set=list(set(relation_list))

In [ ]:
relation_list_set.sort()

In [ ]:
for i in triplelistnew:
    if i.get('triple')[1].lower()=='with cad be':
        print(i)

In [ ]:
df_relation_list_set = pd.DataFrame(relation_list_set)

In [ ]:
df_relation_list_set[:6]

In [ ]:
df_relation_list_set.to_csv('relation_list_set.csv')

relation_list_set[:50]

In [152]:
triplelist[0]

{'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway r

In [177]:
all_es_result=json.load(open('results/all_es_result.json'))

In [178]:
disease_dict_new={}
for i in all_es_result:
    if i.get('type')=='Disease':
        disease_dict_new[i.get('entity')]=True
    if i.get('type')=='Mutation':
        disease_dict_new[i.get('entity')]=True

In [179]:
errorss=[]
for i in triplelist:
    if i.get('target1_type')=='Disease' or i.get('target1_type')=='Mutation':
        if not disease_dict_new.get(i.get('entity1')):
            errorss.append(i.get('entity1'))
    if i.get('target2_type')=='Disease' or i.get('target2_type')=='Mutation':
        if not disease_dict_new.get(i.get('entity2')):
            errorss.append(i.get('entity2'))

In [180]:
len(errorss)

0

In [194]:
triplelistnew[10]

{'triple': ['rs1001179', 'investigated', 'cataract'],
 'PMID': '35556207',
 'sentence': 'Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.',
 'AB': 'Hyperglycemia plays a pronounced role in accelerating the process of aging due to high oxidative stress which triggers dyslipidemia and subsequently led to the progression of cataract. The aim of this study was to investigate lipid profile and its relationship with genotypes of SOD1, GPX1, and CAT variants in cataract patients. Total n = 680 samples were screened in four groups: senile cataract (SC), diabetic cataract (DC), type 2 diabetes mellitus (DM), and controls (CL). Lipid profile was estimated and compared between groups, and its correlation was tested with glycemic markers. Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated i

In [195]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [ ]:
for i in entity_pairs:
    names[i+'_temp']=[]

In [197]:
relation_temp_set=set()
relation_temp_list=[]
relation_temp_type=[]
for i in entity_pairs:
    for j in triplelistnew:
        if (j.get('entity_one_transformed'),j.get('verb'),j.get('entity_two_transformed')) not in relation_temp_set:
            relation_temp_set.add((j.get('entity_one_transformed'),j.get('verb'),j.get('entity_two_transformed')))
            relation_temp_list.append((j.get('entity_one_transformed'),(j.get('entity_two_transformed'))))
            relation_temp_type.append((j.get('entity_one_type').replace(' ','_'),(j.get('entity_two_type').replace(' ','_'))))

In [198]:
len(list(relation_temp_set))

81818

In [205]:
len(relation_temp_list)

81818

In [206]:
relation_temp_list[0]

('SF3B1', 'EIF2AK1')

In [216]:
type_dict_temp={}
for i in range(len(relation_temp_list)):
    # if not type_dict_temp.get((relation_temp_list[i][0],relation_temp_list[i][1])):
    if relation_temp_type[i][0]!=relation_temp_type[i][1]:
        type_dict_temp[(relation_temp_type[i][0],relation_temp_type[i][1])]=type_dict_temp.get((relation_temp_type[i][0],relation_temp_type[i][1]),0)+1
    else:
        type_dict_temp[(relation_temp_type[i][0],'per se')]=type_dict_temp.get((relation_temp_type[i][0],'per se'),0)+1

In [217]:
len(type_dict_temp)

63

In [218]:
type_dict_temp

{('DNA', 'per se'): 2854,
 ('DNA', 'Disease'): 9158,
 ('Mutation', 'Disease'): 458,
 ('Disease', 'per se'): 57009,
 ('Lipid', 'DNA'): 197,
 ('Disease', 'DNA'): 4466,
 ('DNA', 'Lipid'): 233,
 ('DNA', 'Carbohydrate'): 261,
 ('Lipid', 'per se'): 308,
 ('Protein', 'DNA'): 39,
 ('Carbohydrate', 'Disease'): 1183,
 ('Lipid', 'Disease'): 1445,
 ('Mutation', 'Carbohydrate'): 5,
 ('Disease', 'Lipid'): 825,
 ('Protein', 'Disease'): 182,
 ('Disease', 'Mutation'): 292,
 ('Mutation', 'per se'): 86,
 ('Disease', 'Carbohydrate'): 793,
 ('Carbohydrate', 'per se'): 180,
 ('Carbohydrate', 'DNA'): 191,
 ('RNA', 'DNA'): 29,
 ('DNA', 'Protein'): 41,
 ('DNA', 'Peptide'): 53,
 ('DNA', 'Mutation'): 184,
 ('Mutation', 'DNA'): 244,
 ('RNA', 'Disease'): 98,
 ('Disease', 'RNA'): 41,
 ('Lipid', 'Carbohydrate'): 53,
 ('Protein', 'Carbohydrate'): 9,
 ('Disease', 'Protein'): 113,
 ('Mutation', 'Lipid'): 8,
 ('DNA', 'RNA'): 14,
 ('Peptide', 'DNA'): 64,
 ('Peptide', 'Carbohydrate'): 19,
 ('Disease', 'Toxin'): 17,
 ('DNA

In [219]:
type_dict_output={}
for i in range(len(pairs_sorted)-1):
    for j in list(range(0,i))+list(range(i+1,len(pairs_sorted))):
        type_dict_output[(pairs_sorted[i],pairs_sorted[j])]=type_dict_temp.get((pairs_sorted[i],pairs_sorted[j]),0)

In [220]:
pppp=0
for j in type_dict_output.values():
    pppp+=j
print(pppp)

81818


In [226]:
type_list_output=[]
for i,j in type_dict_output.items():
    type_list_output.append(list(i)+[j])

In [221]:
type_dict_output

{('Carbohydrate', 'DNA'): 191,
 ('Carbohydrate', 'Disease'): 1183,
 ('Carbohydrate', 'Lipid'): 54,
 ('Carbohydrate', 'Mutation'): 2,
 ('Carbohydrate', 'Peptide'): 10,
 ('Carbohydrate', 'Pharmaceutical_Preparations'): 0,
 ('Carbohydrate', 'Protein'): 6,
 ('Carbohydrate', 'RNA'): 0,
 ('Carbohydrate', 'Toxin'): 0,
 ('Carbohydrate', 'per se'): 180,
 ('DNA', 'Carbohydrate'): 261,
 ('DNA', 'Disease'): 9158,
 ('DNA', 'Lipid'): 233,
 ('DNA', 'Mutation'): 184,
 ('DNA', 'Peptide'): 53,
 ('DNA', 'Pharmaceutical_Preparations'): 0,
 ('DNA', 'Protein'): 41,
 ('DNA', 'RNA'): 14,
 ('DNA', 'Toxin'): 22,
 ('DNA', 'per se'): 2854,
 ('Disease', 'Carbohydrate'): 793,
 ('Disease', 'DNA'): 4466,
 ('Disease', 'Lipid'): 825,
 ('Disease', 'Mutation'): 292,
 ('Disease', 'Peptide'): 152,
 ('Disease', 'Pharmaceutical_Preparations'): 10,
 ('Disease', 'Protein'): 113,
 ('Disease', 'RNA'): 41,
 ('Disease', 'Toxin'): 17,
 ('Disease', 'per se'): 57009,
 ('Lipid', 'Carbohydrate'): 53,
 ('Lipid', 'DNA'): 197,
 ('Lipid', 

In [224]:
for i,j in type_dict_output.items():
    if i[1]=='Disease':
        print(i[0]+' '+str(j))
    elif i[0]=='Disease' and i[1]=='per se':
        print(i[0]+' '+str(j))

Carbohydrate 1183
DNA 9158
Disease 57009
Lipid 1445
Mutation 458
Peptide 300
Pharmaceutical_Preparations 16
Protein 182
RNA 98
Toxin 15


In [227]:
sorted(type_list_output)

[['Carbohydrate', 'DNA', 191],
 ['Carbohydrate', 'Disease', 1183],
 ['Carbohydrate', 'Lipid', 54],
 ['Carbohydrate', 'Mutation', 2],
 ['Carbohydrate', 'Peptide', 10],
 ['Carbohydrate', 'Pharmaceutical_Preparations', 0],
 ['Carbohydrate', 'Protein', 6],
 ['Carbohydrate', 'RNA', 0],
 ['Carbohydrate', 'Toxin', 0],
 ['Carbohydrate', 'per se', 180],
 ['DNA', 'Carbohydrate', 261],
 ['DNA', 'Disease', 9158],
 ['DNA', 'Lipid', 233],
 ['DNA', 'Mutation', 184],
 ['DNA', 'Peptide', 53],
 ['DNA', 'Pharmaceutical_Preparations', 0],
 ['DNA', 'Protein', 41],
 ['DNA', 'RNA', 14],
 ['DNA', 'Toxin', 22],
 ['DNA', 'per se', 2854],
 ['Disease', 'Carbohydrate', 793],
 ['Disease', 'DNA', 4466],
 ['Disease', 'Lipid', 825],
 ['Disease', 'Mutation', 292],
 ['Disease', 'Peptide', 152],
 ['Disease', 'Pharmaceutical_Preparations', 10],
 ['Disease', 'Protein', 113],
 ['Disease', 'RNA', 41],
 ['Disease', 'Toxin', 17],
 ['Disease', 'per se', 57009],
 ['Lipid', 'Carbohydrate', 53],
 ['Lipid', 'DNA', 197],
 ['Lipid', 

In [150]:
triplelist[0]

{'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway r

In [152]:
for i in triplelist:
    if i.get('entity2')=='apolipoprotein E':
        print(i)
        break

In [165]:
disease_list[0]

{'triple': ['SF3B1', 'lead', 'heme deficiency'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this path

In [411]:
for i in disease_list:
    if i.get('triple')==['KL', 'may serve as', 'colon cancer']:
        print(i)

In [447]:
for i in disease_list:
    if i.get('verb')=='yield':
        print(i)
        # break

{'triple': ['Tet2', 'yield', 'CMML'], 'PMID': '36271153', 'sentence': 'We reproduce Srsf2/Tet2 co-operativity in vivo, yielding a disease with core characteristics of CMML, unlike single Srsf2 or Tet2 mutation.', 'AB': 'Recurrent mutations in RNA splicing proteins and epigenetic regulators contribute to the development of myelodysplastic syndrome (MDS) and related myeloid neoplasms. In chronic myelomonocytic leukemia (CMML), SRSF2 mutations occur in ~50% of patients and TET2 mutations in ~60%. Clonal analysis indicates that either mutation can arise as the founder lesion. Based on human cancer genetics we crossed an inducible Srsf2P95H/+ mutant model with Tet2fl/fl mice to mutate both concomitantly in hematopoietic stem cells. At 20-24 weeks post mutation induction, we observed subtle differences in the Srsf2/Tet2 mutants compared to either single mutant. Under conditions of native hematopoiesis with aging, we see a distinct myeloid bias and monocytosis in the Srsf2/Tet2 mutants. A sub

In [304]:
for i in triplelist:
    if i.get('PMID')=='22197663' and i.get('sentence')=='The dysfunction has involved insulin resistance (IR), which occurs in most women with PCOS.':
        print(i)

{'PMID': '22197663', 'entity1': 'Polycystic Ovary Syndrome', 'entity2': 'Neurologic Manifestations', 'sentence': 'The dysfunction has involved insulin resistance (IR), which occurs in most women with PCOS.', 'AB': 'Life-long apolipoprotein lipid metabolic dysfunction in women with PCOS exaggerates the risk for cardiovascular disease (CVD) with aging. The dysfunction has involved insulin resistance (IR), which occurs in most women with PCOS. Women with PCOS have androgen excess, IR, variable amounts of estrogen exposure, and many environmental factors, all of which can influence lipid metabolism. On average, women with PCOS were higher triglyceride [26.39 95% CI (17.24, 35.54)], lower HDL-cholesterol [6.41 95% CI (3.69, 9.14)], and higher non HDL cholesterol levels [18.82 95% CI (15.53, 22.11)] than their non-PCOS counterparts. They have higher ApoCIII/ApoCII ratios and higher ApoCI even if they are not obese. ApoC1 elevation in women with PCOS needs to be further evaluated as a marker 

In [183]:
for i in triplelist:
    if i.get('target2')=='mortality' and i.get('target1')=='digoxin':
        print(i)

{'PMID': '11190416', 'entity1': 'Digoxin', 'entity2': 'Death', 'sentence': 'Greater knowledge about the causes and prevention of digoxin toxicity should further reduce the morbidity and mortality arising from digoxin toxicity, especially in the elderly population.', 'AB': 'Digoxin remains one of the most commonly prescribed of all cardiac medications. The main indications for digoxin usage include atrial fibrillation and heart failure; both these conditions are more prevalent in older patients. Given the aging population and the increasing incidence of heart failure we would expect prescribing of digoxin to remain as frequent or to even increase in older patients. Older patients are also more likely to develop toxicity and diagnosis of digoxin toxicity can be difficult in this group. Numerous components contribute to the development of toxicity in older patients, ranging from aging-related changes in renal function or body mass to polypharmacy and possible interactions with digoxin. It

In [181]:
triplelist[0]

{'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway r